# KNN & Distance Metrics

## Part 1 - Necessary Code

In [1]:
import json

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.neighbors import KNeighborsClassifier
from sklearn.mixture import GaussianMixture

from sklearn.metrics import accuracy_score

### Import company colors & data

In [2]:
# Import company colors
with open('plot_colors.json', 'r') as pc:
    color_dict = json.load(pc)

c_light, c_dark, c_blue = color_dict['color_light'], color_dict['color_dark'], color_dict['color_blue']

In [3]:
# Import data
df = pd.read_csv("../data/knn_data.txt")
X_train = df[['x1', 'x2']].values
y_train = df.y.values

### Plot data & decision boundary 

In [4]:
# Predict for different k a mesh
def fill_prediction_grid(X_train, y_train, k, n1, n2):
    # Train model with k
    knn = KNeighborsClassifier(n_neighbors=k).fit(X_train, y_train)
    
    # Create mesh with predictions
    x1, x2 = np.linspace(-3, 4.5, n1), np.linspace(-2.5, 3, n2)
    X = np.transpose([np.tile(x1, n2), np.repeat(x2, n1)])
    y = knn.predict(X)
    X0 = X[:, 0].reshape(n1, n2)
    X1 = X[:, 1].reshape(n1, n2)
    y_reshape = y.reshape(n1, n2)
    return X, y, X0, X1, y_reshape


def plot_decision_boundary(X_train, y_train, k, title, figsize=(10,6), ax=None):
    if ax==None:
        fig, ax = plt.subplots(figsize=figsize)
        
    ax.set_aspect(1.3)    
    
    # Plot training data
    ax.scatter(X_train[:, 0], X_train[:, 1], s=20, facecolors='none',
               edgecolors=np.array([c_light, c_blue])[y_train])
    
    # Plot background dots
    X, y, X0, X1, y_reshape = fill_prediction_grid(X_train, y_train, k, 100, 100)
    ax.scatter(X[:, 0], X[:, 1], marker='.', lw=0, s=2,
               c=np.array([c_light, c_blue])[y])
    
    # Plot decision boundary
    ax.contour(X0, X1, y_reshape, [0.5], colors=c_dark, linewidths=[0.7])
    ax.set_title(title)
    ax.set_xticks([])
    ax.set_yticks([])

## Part 2 - Lecture

<div class="slide-title"> 
        
# KNN & Distance Metrics
            
</div> 

## Recap on general concepts

### What’s the difference between supervised and unsupervised learning?

<center>
    <img src="../images/knn_distance_metrics/img_p4_2.png">
    Supervised Learning vs. Unsupervised Learning
</center>

Notes: Supervised Learning: Supervised Learning: Supervised learning is a machine learning approach that’s defined by its use of labeled datasets. These datasets are designed to train or “supervise” algorithms into classifying data or predicting outcomes accurately. Using labeled inputs and outputs, the model can measure its accuracy and learn over time.


Unsupervised Learning: Unsupervised learning uses machine learning algorithms to analyze and cluster unlabeled data sets. These algorithms discover hidden patterns in data without the need for human intervention (hence, they are “unsupervised”).

source: https://www.ibm.com/cloud/blog/supervised-vs-unsupervised-learning

### What are the two different tasks that can be solved with supervised learning?



<div class="group">
  <div class="images">  
    <img src="../images/knn_distance_metrics/img_p6_2.png" width="400">
  </div>
  <div class="images">
    <img src="../images/knn_distance_metrics/img_p6_3.png" width="350">
  </div>
</div>

Notes: Regression: the regression model is trained in such a way that it predicts continuous numerical value as an output based on input variables.
The algorithm maps the input data (x) to continuous or numerical data(y).

Classification: In classification, the model is trained in such a way that the output data is separated into different labels (or categories) according to the given input data.
The algorithm maps the input data (x) to discrete labels (y).
source: https://medium.com/swlh/what-are-classification-and-regression-3677987b9422

## The KNN Algorithm

### Overview of KNN

<div class="group">
  <div class="text_70">
      
- KNN is a supervised learning algorithm
      
- used to solve both regression and classification problems
      
- KNN assumes that similar things exist in close proximity
      
→ “Birds of a feather flock together”
  </div>
  <div class="images_30">
    <img src="../images/knn_distance_metrics/img_p8_1.png" width="450">
  </div>

</div>

### Why using KNN?
- Despite its simplicity, it was often successful for both classification and regression problems:
    - handwritten digits
    - satellite image scenes
    
- it’s good for situations where decision boundary is very irregular

Notes: Despite its simplicity, nearest neighbors has been successful in a large number of classification and regression problems, including handwritten digits and satellite image scenes. Being a non-parametric method, it is often successful in classification situations where the decision boundary is very irregular.

source: https://scikit-learn.org/stable/modules/neighbors.html#nearest-neighbors

### How would you classify the question mark?

<center>
<img src="../images/knn_distance_metrics/img_p10_1.png" width="800">
</center>

Notes: probably they will say cat… then ask WHY cat?

### How to measure “close proximity”?
- KNN is based on the idea of **“similarity”**
- mathematically speaking similarity can be calculated via **distances**
- simplest distance would be bee line distance → Euclidean Distance

<div class="alert alert-block alert-warning">
Which features will cause problems for calculating distances?
What could you do against it?
</div>

Notes: Which features will cause problems for calculating distances?
categorical features
What could you do against it?
depends if they can be ranked or in another way be transformed to numbers, otherwise dummy-encode 

### KNN
Input:
- data with features that are comparable (i.e. can calculate a distance) & **target variable**

How KNN works:
- loop over the observations:
- calculate distance to all (brute force) or some (optimized) data points
- **sort** them and pick K “closest”

<div class="alert alert-block alert-warning">
What would be the output for Regression and Classification?
</div>



Notes: regression: mean
classification: mode

Output:
- Regression: the **mean** of the neighbors
- Classification: the **mode** of the neighbors



### How to train KNN?


<div class="group">
  <div class="text">
      
What’s happening when we call `knn.fit()`:
- training phase consists of “remembering” / storing all data points

→ training time is very short
  </div>
  <div class="images">
    <img src="../images/knn_distance_metrics/img_p14_1.png" width="450">
  </div>

</div>



Notes: thats why KNN is also called a “lazy algorithm”
on the right:
binary classification problem
our training data 

### How to predict for new instances?


<div class="group">
  <div class="text">
      
- <b>calculate distance</b> between new observations and every
training data point
- <b>K closest points</b> are determined
- test data are assigned to the class of their K nearest
neighbors according to <b>majority voting</b>
      
→ prediction time can be very high
  </div>
  <div class="images">
    <img src="../images/knn_distance_metrics/img_p15_1.png" width="400">
  </div>

</div>


<div class="alert alert-block alert-warning">

What would you predict for k = 3 and k =6 ?
</div>



### How to predict for new instances?


<div class="group">
  <div class="text">
      
<b>k = 3:</b>
      
1 neighbor is orange (p=⅓)
      
2 neighbors are blue (p=⅔)
      
→ green point belongs to <b>class B</b>
      
<b>k = 6:</b>
      
4 neighbors are orange ( p=⅔)
      
2 neighbors are blue (p=⅓)
      
→ green point belongs to <b>class A</b>
      
  </div>
  <div class="images">
    <img src="../images/knn_distance_metrics/img_p15_1.png" width="400">
  </div>

</div>


### What are the assumptions of KNN?


<div class="group">
  <div class="text">
      
- similar observations belong to the same class / have a
similar outcome
      
- no assumption is made on data distribution / mapping
function
      
→ KNN is a <b>non-parametric algorithm</b>
  </div>
  <div class="images">
    <img src="../images/knn_distance_metrics/img_p17_1.png" width="400">
  </div>

</div>


<div class="alert alert-block alert-warning">

What are some assumptions of linear regression?

</div>






Notes: For example assumptions of lin reg:
Linearity: linear regression assumes our response and explanatory variables are linearly related.
Normality: The residuals of the model should follow a normal distribution.
Independence: Observations are independent of each other.
Homoscedasticity: The variance of residual is the same for any value of X. 

## Hyperparameters

### What influences the performance of KNN?

### What are hyperparameters of KNN?
1. The **number of neighbors (K)** which is taken to classify.
2. The **distance metric** used to determine the nearest neighbors.
3. The **weight** applied to neighbours during prediction phase.


### Influence of K

<div class="group">
  <div class="text">
      
- **circles**: training data
- **dashed line**: decision boundary of data generating process
- **solid line**: model
- **grid points**: indicating which class is predicted in this area
      
  </div>
  <div class="images">
    <img src="../images/knn_distance_metrics/img_p20_1.png" width="400">
  </div>
</div>


### Influence of K

How would you interpret these models? 

<center>
    <img src="../images/knn_distance_metrics/k_1_10_100.png">
</center>

Notes: plot info: 
data generating process: is the dashed decision boundary
training points: the circles
test data not shown but the yellow and blue grid (mini dots) represent the predicted classes

interpretation:
overfitting (small k value) → taking into consideration highly local data, creating even encapsuled small “islands” of blue in yellow class, highly dependent on given train data (high variance)
perfect fit (middle k) → pretty close to the truth (data generating process of the dashed decision boundary)
underfitting (high k value) → non flexible decision boundary, locally biased decisions (esp in areas of s-shape)

### Influence of K

<div class="group">
  <div class="column">
      
K=1 → overfitting
      
  </div>
  <div class="column">
      
K=10 → good fit
      
  </div>
  <div class="column">
      
K=100 → underfitting
  </div>
</div>  

<center>
    <img src="../images/knn_distance_metrics/k_1_10_100.png">
</center>

### How do we determine the best value for K?

<div class="group">
  <div class="text">
      
- test different numbers for K (e.g. with GridSearch)
      
→ take K with **lowest error on validation data set**
      
  </div>
  <div class="images">
    <img src="../images/knn_distance_metrics/img_p20_1.png" width="400">
  </div>

</div>

## Distance Metrics

<div class="group">
  <div class="text">
      
In this example the <b>Euclidean Distance</b> is used.
      
The Euclidean distance between two points ($a_1$ and $a_2$ ) is the length of the path connecting them.
      
$\sqrt{\Sigma_{i=1}^{n}(a_{1,i} - a_{2,i})^2}$

      
  </div>
  <div class="images">
    <img src="../images/knn_distance_metrics/img_p15_1.png" width="400">
  </div>
</div>

<div class="alert alert-block alert-info">
n = number of features
</div>

### What Distance Metrics exist?

<div class="group">
  <div class="text">

<font color ="green">Euclidean Distance</font> $\sqrt{\Sigma_{i=1}^{n}(a_{1,i} - a_{2,i})^2}$

<font color ="red">Manhattan Distance</font> $\Sigma_{i=1}^{n}|a_{1,i} - a_{2,i}|$
      
sum of the absolute differences between the &emsp; x-coordinates and y-coordinates

(Minkowski Distance $\sqrt[p]{\Sigma_{i=1}^{n}|a_{1,i} - a_{2,i}|^p}$)
     
  </div>
  <div class="images">
    <img src="../images/knn_distance_metrics/img_p26_2.png" width="400">
  </div>

</div>

Notes: n: number of dimensions/features 
Manhatten: sum of absolute differences between points across all the dimensions
Minkowski:  generalized form of Euclidean and Manhattan Distance. (p=3, Minkowski Distance of the order 3)

### Minkowski Distance, mostly used cases

**Minkowski Distance:** &emsp;&emsp;&emsp;&emsp;$ \sqrt[p]{\Sigma_{i=1}^{n}|a_{1,i} - a_{2,i}|^p}$&emsp;&emsp;&emsp;&emsp;generalized Distance Metric

**Minkowski Distance with p=1**: &emsp;&emsp;&emsp;&emsp;$ \Sigma_{i=1}^{n}|a_{1,i} - a_{2,i}|$&emsp;&emsp;&emsp;&emsp;&emsp;<font color ="red">Manhattan Distance</font>

**Minkowski Distance with p=2**:&emsp;&emsp;&emsp;&emsp;
$ \sqrt{\Sigma_{i=1}^{n}(a_{1,i} - a_{2,i})^2}$&emsp;&emsp;&emsp;<font color ="green">Euclidean Distance</font>

**Minkowski Distance with p=∞**: &emsp;&emsp;&emsp;&emsp;
$ \sqrt[∞]{\Sigma_{i=1}^{n}|a_{1,i} - a_{2,i}|^∞}$&emsp;&emsp;&emsp;Chebychev Distance


Notes: Chebychev: "maximum metric" in mathematics, measures distance between two points as the maximum difference over any of their axis values.

### Cosine Distance


COSINE SIMILARITY: the measure of similarity between two non-zero vectors defined in an inner product space. Cosine similarity is the cosine of the angle between the vectors.

**Cosine Similarity**: &emsp;&emsp;&emsp;&emsp;
$ S_c(\vec A,\vec B)= cos(\Theta)= { \vec A \cdot \vec B} \div {|\vec A| |\vec B|} $&emsp;&emsp;&emsp;

**COSINE DISTANCE** =&emsp;&emsp;&emsp;&emsp; 
$ 1 -  S_c(\vec A,\vec B) $


  </div>
  <div class="images">
    <img src="../images/knn_distance_metrics/cosine_distance.png" width="400">
  </div>

</div>

source: https://pub.aimind.so/understanding-cosine-similarity-and-cosine-distance-in-depth-cc91eac3ef2

### When to use which Distance Metric?

**Euclidean Distance** (Minkowski p=2 / L2-norm): most commonly used; when data is dense or
continuous, this is the best proximity measure.

**Manhattan Distance** (Minkowski p=1 / L1-norm, Taxicab or Cityblock distance):
Better for sparse data (high dimensional data).

**→ Whenever Distance Metrics are used, keep in mind to scale the data**

### Applications of Distance Metrics
<p></p>

- KNN

- K-means clustering

- NLP

Notes: Do you remember other reasons for scaling your data?
gradient descent: will faster find a local minima if all features are on same scale
if you want to compare the influence of features in a linear/logistic regression model == interpret/compare the values of the coefficients (bs), make sure all features are on the same scale

## References


<p></p>
<div class="group">
  <div class="text">
- Hastie, T., Hastie, T., Tibshirani, R., & Friedman, J. H.
(2001). The elements of statistical learning: Data
mining, inference, and prediction. New York:
Springer.
  </div>
  <div class="images">
    <img src="../images/knn_distance_metrics/img_p30_1.png" width="400">
  </div>

</div>